# Analytical solution

In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

In [ ]:
#Define the Input Values

#define the value of knowns
u=3
k=1
f=1

#Define a range of x_values
x_start=0
x_end=1

#Define boundary conditions
T_start=0
T_end=0

# Number of nodes and elements
num_elements = 1024
num_nodes = num_elements+1



In [ ]:
# Define the variable and the function
x = sp.symbols('x')
T = sp.Function('T')(x)

# Define the total derivative
dT_dx = T.diff(x)
d2T_dx2 = T.diff(x, 2)

# Create the equation using total derivatives
pde = sp.Eq(u * dT_dx - k*d2T_dx2, f)

# Display the equation
#sp.pretty(pde)

# Solve the ODE with initial and final conditions
sol = sp.dsolve(pde, ics={T.subs(x, x_start): T_start, T.subs(x, x_end): T_end})  # Adjust the initial and final conditions as needed

# Define the range of x values
x_values = np.linspace(x_start, x_end, 100)  # Adjust the range and number of points as needed

# Substitute the solution into the expression for T
T_values = [sol.rhs.subs(x, val).evalf() for val in x_values]


# create a local stiffness matrix

In [ ]:
# Define the variable and the functions
zi = sp.symbols('zi')
S1 = sp.Function('S1')(zi)
S2 = sp.Function('S2')(zi)

# Define shape functions
S1 = (1 - zi) / 2
S2 = (1 + zi) / 2

shape_functions=np.array([S1,S2])
# Define the total derivative
dS1_dzi = S1.diff(zi)
dS2_dzi = S2.diff(zi)

# Define Je
he=1/num_elements
Je=he/2

# Define the combinations of shape functions
combinations = [(S1, S1), (S1, S2), (S2, S1), (S2, S2)]

# Initialize a dictionary to store the integration results
integration_results = np.array([])

# Perform integration for each combination
for combination in combinations:
    S1_comb, S2_comb = combination
    
    # Define the function to be integrated based on the combination
    def f(zi_value):
        S1_value = S1_comb.subs(zi, zi_value)
        dS1_dzi_value = S1_comb.diff(zi).subs(zi, zi_value)
        dS2_dzi_value = S2_comb.diff(zi).subs(zi, zi_value)
        return (S1_value * u * dS2_dzi_value * (1 / Je) + k * (dS1_dzi_value) * (1 / Je) * (dS2_dzi_value) * (1 / Je)) * Je
    
    # Use integrate.quad with the numerical function
    result, error = integrate.quad(f, -1, 1)
    
    # Store the result in the dictionary
    integration_results = np.append(integration_results,np.round(result,1))

print(integration_results)

# Assemble a global stiffness matrix

In [ ]:
# Initialize the global stiffness matrix
K_global = np.zeros((num_nodes, num_nodes))

# Define the local stiffness matrices for each element (assuming 2x2 matrices)
K_element_1 = np.array([[integration_results[0],integration_results[1] ],
                         [integration_results[2],integration_results[3]]])  # Replace k11, k12, k21, k22 with actual values

K_element_2 = np.array([[integration_results[0],integration_results[1] ],
                         [integration_results[2],integration_results[3]]])


# Define the connectivity of elements to nodes
def create_element_node_connectivity(num_elements):
    element_node_connectivity={}
    for i in range(1,num_elements+1):
        element_node_connectivity[i]=[i,i+1]
        
    return element_node_connectivity 

element_node_connectivity=create_element_node_connectivity(num_elements)

# Assemble the local stiffness matrices into the global stiffness matrix
for element in range(1, num_elements + 1):
    # Get the nodes associated with the current element
    nodes = element_node_connectivity[element]

    # Add the contributions of the local stiffness matrix to the global stiffness matrix
    for i, node_i in enumerate(nodes):
        for j, node_j in enumerate(nodes):
            # Map local degrees of freedom to global degrees of freedom
            global_dof_i = node_i - 1  
            global_dof_j = node_j - 1  

            # Add the contribution of the local stiffness matrix to the global stiffness matrix
            if node_i==node_j and node_i>1 and node_i<num_nodes:
                K_global[global_dof_i, global_dof_j] = K_element_1[i, j] + K_element_2[i-1, j-1]
            else:
                K_global[global_dof_i, global_dof_j] = K_element_1[i, j]
                

# Print the global stiffness matrix
print("Global Stiffness Matrix:")
print(K_global)

# Create a F matrix 

In [ ]:
F_matrix = []
for one_shape_function in shape_functions:
    expression = one_shape_function * 1 * Je
    definite_integral_result = sp.integrate(expression, (zi, -1, 1))
    F_matrix.append(definite_integral_result)

F_global = np.zeros((num_nodes, 1))

for i in range(0,num_nodes):
    if i==0:
        F_global[i,0]=F_matrix[0]
    elif i==num_nodes-1:
        F_global[i,0]=F_matrix[0]
    else:
        F_global[i,0]=F_matrix[0]+F_matrix[1]
        
print(F_global)
    


# Perform Matrix reduction 

In [ ]:
T = np.array([T_start] + [f'T{i}' for i in range(1, num_nodes-1)] + [T_end])

B=[]

for i in range(1,num_nodes+1):
    if i==1:
        B=np.append(B,f'B{i}')
    elif i==num_nodes:
        B=np.append(B,f'B{i}')
    else:
        B=np.append(B,0)

non_zero_indices=np.array([],dtype= int)


for i, value in enumerate(T):
    if T[i]=='0':
        continue
    else:
        non_zero_indices=np.append(non_zero_indices,i)

length=len(non_zero_indices)


filtered_F=np.array([])
for i in non_zero_indices:
    filtered_F=np.append(filtered_F,F_global[i,0])

filtered_F=filtered_F.reshape(length,1)

filtered_B=np.array([])
for i in non_zero_indices:
    filtered_B=np.append(filtered_B,B[i])

filtered_B=filtered_B.reshape(length,1).astype(int)

filtered_K=np.array([])
for i in non_zero_indices:
    for j in non_zero_indices:
        filtered_K=np.append(filtered_K,K_global[i,j])

filtered_K=filtered_K.reshape(length,length)

print(non_zero_indices)

# Calculate the unknowns 

In [ ]:
# Calculate the inverse of matrix k
k_inverse = np.linalg.inv(filtered_K)


# Calculate vector t using the formula t = k^(-1) * (a + b)
t = np.dot(k_inverse, filtered_F+filtered_B)

# Print the result
print("Vector t is:")
print(t)

# Plot the result 

In [ ]:
t= np.insert(t, 0, T_start)
t= np.insert(t, num_nodes-1, T_end)
x=np.linspace(0,1,num_nodes)


# Plot T against x
plt.figure(figsize=(8, 6))
plt.plot(x_values, T_values, label='T(x)-ANALYTICAL')
plt.plot(x, t, label='T(x)-FEM')
plt.xlabel('x')
plt.ylabel('T(x)')
plt.title('Plot of T(x)')
plt.grid(True)
plt.legend()
plt.show()